Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [3]:
# подгружаем .env
load_dotenv()

True

In [4]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [5]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [6]:
from sqlalchemy import inspect
inspector = inspect(dst_conn)

for table_name in inspector.get_table_names():
   print(table_name)

alt_users_churn
users_churn


In [7]:
# Пример выгрузки данных из БД
SQL = f'select * from users_churn'
data = pd.read_sql(SQL, dst_conn)
data

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,6391,6994-FGRHH,2014-04-01,NaT,One year,Yes,Electronic check,106.15,7475.10,Fiber optic,...,Yes,Yes,Yes,Yes,Male,0,Yes,Yes,No,0
1,4764,4001-TSBTV,2015-04-01,NaT,Month-to-month,Yes,Electronic check,91.55,5511.65,Fiber optic,...,Yes,No,Yes,No,Female,0,Yes,Yes,No,0
2,6448,8782-NUUOL,2015-02-01,NaT,One year,No,Mailed check,79.00,4801.10,DSL,...,Yes,Yes,Yes,Yes,Male,0,No,No,No,0
3,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,None,0
4,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,7039,6840-RESVB,2018-02-01,NaT,One year,Yes,Mailed check,84.80,1990.50,DSL,...,Yes,Yes,Yes,Yes,Male,0,Yes,Yes,Yes,0
7039,7040,2234-XADUH,2014-02-01,NaT,One year,Yes,Credit card (automatic),103.20,7362.90,Fiber optic,...,Yes,No,Yes,Yes,Female,0,Yes,Yes,Yes,0
7040,7041,4801-JZAZL,2019-03-01,NaT,Month-to-month,Yes,Electronic check,29.60,346.45,DSL,...,No,No,No,No,Female,0,Yes,Yes,None,0
7041,7042,8361-LTMKD,2019-07-01,2019-11-01,Month-to-month,Yes,Mailed check,74.40,306.60,Fiber optic,...,No,No,No,No,Male,1,Yes,No,Yes,1


In [19]:
data['device_protection'].mode().iloc[0]

'No'

In [14]:
data.columns.drop('customer_id')

Index(['id', 'begin_date', 'end_date', 'type', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges',
       'internet_service', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'gender', 'senior_citizen', 'partner', 'dependents', 'multiple_lines',
       'target'],
      dtype='object')

In [9]:
data.duplicated(keep=False)

0       False
1       False
2       False
3       False
4       False
        ...  
7038    False
7039    False
7040    False
7041    False
7042    False
Length: 7043, dtype: bool

In [7]:
features = data.drop(['customer_id', 'target'], axis=1)
features.dtypes.value_counts()

object            14
int64              2
datetime64[ns]     2
float64            2
Name: count, dtype: int64

In [8]:
num_features = features.select_dtypes(include=['float', 'int'])
date_features = features.select_dtypes(include='datetime64[ns]')
cat_features = features.select_dtypes(include='object')

In [18]:
unique_values_per_col = cat_features.nunique()
unique_values_per_col

type                 3
paperless_billing    2
payment_method       4
internet_service     2
online_security      2
online_backup        2
device_protection    2
tech_support         2
streaming_tv         2
streaming_movies     2
gender               2
partner              2
dependents           2
multiple_lines       2
dtype: int64

In [19]:
unique_values_per_col.value_counts()

2    12
3     1
4     1
Name: count, dtype: int64

In [25]:
unique_values_per_col = cat_features.nunique()
binary_cat_features = cat_features[unique_values_per_col[unique_values_per_col == 2].index]
other_cat_features = cat_features[unique_values_per_col[unique_values_per_col != 2].index]

In [34]:
yes_no_features = binary_cat_features.isin(['Yes', 'No', None]).all()
binary_cat_features[yes_no_features[yes_no_features == True].index]

,paperless_billing,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,partner,dependents,multiple_lines
0,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No
1,Yes,No,Yes,Yes,No,Yes,No,Yes,Yes,No
2,No,Yes,No,Yes,Yes,Yes,Yes,No,No,No
3,Yes,No,Yes,No,No,No,No,Yes,No,None
4,No,Yes,No,Yes,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...
7038,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes
7039,Yes,No,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes
7040,Yes,Yes,No,No,No,No,No,Yes,Yes,None
7041,Yes,No,No,No,No,No,No,Yes,No,Yes


In [27]:
src_conn.dispose()
dst_conn.dispose()

А дальше, творите!